In [435]:
import json
import toml

ObjectInspector.py

In [436]:
import builtins
import inspect
import functools
import types
import dis
import re

primitives = set(
    [
        int,
        float,
        bool,
        str
    ])

def is_magicmarked(s: str) -> bool:
    return re.match("^__(?:\w+)__$", s) != None

def is_primitive(obj: object) -> bool:
    return type(obj) in primitives

def is_basetype(obj: object) -> bool:
    for el in primitives:
        if el.__name__ == obj.__name__:
            return True
    if el in [dict, list, tuple, set]:
        if el.__name__ == obj.__name__:
            return True
    return False

def is_instance(obj):
    if not hasattr(obj, '__dict__'):
        return False
    if inspect.isroutine(obj): 
        return False
    if inspect.isclass(obj):
        return False
    else:
        return True

def is_none(obj: object) -> bool:
    return obj is None
    
def fetch_typereferences(cls):
    if inspect.isclass(cls):
        mro = inspect.getmro(cls)
        metamro = inspect.getmro(type(cls))
        metamro = tuple(cls for cls in metamro if cls not in (type, object))
        class_bases = mro
        if not type in mro and len(metamro) != 0:
            return class_bases[1:-1], metamro[0]
        else:
            return class_bases[1:-1], None
            
def fetch_funcreferences(func: object):
    if inspect.ismethod(func):
        func = func.__func__

    if not inspect.isfunction(func):
        raise TypeError("{!r} is not a Python function".format(func))

    code = func.__code__
    if func.__closure__ is None:
        nonlocal_vars = {}
    else:
        nonlocal_vars = {
            var : cell.cell_contents
            for var, cell in zip(code.co_freevars, func.__closure__)
       }

    global_ns = func.__globals__
    builtin_ns = global_ns.get("__builtins__", builtins.__dict__)
    if inspect.ismodule(builtin_ns):
        builtin_ns = builtin_ns.__dict__
    global_vars = {}
    builtin_vars = {}
    unbound_names = set()
    for name in code.co_names:
        if name in ("None", "True", "False"):
            continue
        try:
            global_vars[name] = global_ns[name]
        except KeyError:
            try:
                builtin_vars[name] = builtin_ns[name]
            except KeyError:
                unbound_names.add(name)

    return (nonlocal_vars, global_vars,
                       builtin_vars, unbound_names)
            
def deconstruct_class(cls):
    attributes = inspect.classify_class_attrs(cls)
    deconstructed = []
    for attr in attributes:
        if attr.defining_class == object or attr.defining_class == type or attr.name in ["__dict__", "__weakref__"]:
            continue
        else:
            deconstructed.append((
                attr.name,
                attr.object,
                attr.kind
            ))
    return deconstructed

def deconstruct_func(func):
    code = {el: getattr(func.__code__, el) for el in func.__code__.__dir__() if not is_magicmarked(el) and "co" in el}
    
    refs = fetch_funcreferences(func)
    defaults = func.__defaults__
    return {'.name': func.__name__, '.code': code, '.references': refs, '.defaults': defaults}

def getfields(obj):
    """Try to get as much attributes as possible"""
    members = inspect.getmembers(obj)
    cls = type(obj)
    type_attrnames = [el.name for el in inspect.classify_class_attrs(cls)]
    result = {}
    
    for member in members:
        if not member[0] in type_attrnames:
            result[member[0]] = member[1]
            
    return result

def deconstruct_instance(obj):
    type_ = type(obj)
    fields = getfields(obj)
    
    return (type_, fields)



Data samples

In [437]:
globala = "Hui"

class MetaS(type):
    def __new__(cls, *args):
        obj = super(cls, cls).__new__(cls, *args)
        return obj

class Parent:
    def _getAttreasdasd(self):
        print(globala +  "Asdas")
    
class ClassA(Parent,metaclass=MetaS):
    a = range
    def __init__(self):
        self.dict = json.dumps({"asd": 1})

def func1(name):
    print(globala + name)

In [438]:
import inspect

class AbstractMetaclass():
    @staticmethod
    def create(name, mro):
        globals().update({el.__name__:el for el in mro[0]})
        if len(mro[0]) != 0:
            bases = ",".join([base.__name__ for base in mro[0]])
        else:
            bases = ""
        exec(f"class {name}({bases}):\n\tpass")
        metaclass = eval(f"{name}")
        return metaclass

class AbstractClass():
    @staticmethod
    def create(name, mro):
        globals().update({el.__name__:el for el in mro[0]})
        globals().update({mro[1].__name__:mro[1]})
        if len(mro[0]) != 0:
            bases = ",".join([base.__name__ for base in mro[0]])
        else:
            bases = ""
        if mro[1]:
            meta = "metaclass="+mro[1].__name__
        else:
            meta = ""
        if bases != "":
            str_ = bases + ", "+ meta
        else:
            str_ = meta
        exec(f"class {name}({str_}):\n\tpass")
        _class = eval(f"{name}")
        return _class

def create_classbase(name, mro=None):
    if mro[1]:
        template = AbstractClass.create(name, mro)
    else: 
        template = AbstractMetaclass.create(name, mro)
    return template

def set_classattrs(cls, attributes=None):
    if attributes:
        for el in attributes:
            if el[1] != None:
                try:
                    if el[2] == "static method":
                        setattr(cls, el[0], staticmethod(el[1]))
                    elif el[2] == "class method":
                        setattr(cls, el[0], classmethod(el[1]))
                    else:
                        setattr(cls, el[0], el[1])
                except AttributeError:
                    continue
    return cls
    
    
def create_class(name, mro=None, attributes=None):
    if mro[1]:
        template = AbstractClass.create(name, mro)
    else: 
        template = AbstractMetaclass.create(name, mro)
    if attributes:
        for el in attributes:
            if el[0] == "__dict__" or el[0] == "__weakref__":
                continue
            if el[1] != None:
                try:
                    if el[2] == "static method":
                        setattr(template, el[0], staticmethod(el[1]))
                    elif el[2] == "class method":
                        setattr(template, el[0], classmethod(el[1]))
                    else:
                        setattr(template, el[0], el[1])
                except AttributeError:
                    continue
    return template
    
def create_instance(type_, fields):
    instance = type_.__new__(type_)
    for el in fields:
        setattr(instance, el, fields[el])
    return instance

def cell_factory(el):
    inner = el
    def _f():
        return el
    return _f.__closure__[0]

def get_code(obj):
    lines = inspect.getsourcelines(obj)[0]
    tabs = 0
    for ch in lines[0]:
        if ch == ' ':
            tabs += 1
        else:
            break
    new_lines = []
    for line in lines:
        if len(line) >= tabs:
            line = line[tabs:]
        else:
            pass
        new_lines.append(line)
    return "\n".join(new_lines)

In [439]:
NewClassA = create_class("ClassA", mro=fetch_typereferences(ClassA), attributes=deconstruct_class(ClassA))

print(dir(ClassA))

print(dir(NewClassA))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_getAttreasdasd', 'a']
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_getAttreasdasd', 'a']


In [440]:
for _ in NewClassA.a(12):
    print("AHAHHA")

AHAHHA
AHAHHA
AHAHHA
AHAHHA
AHAHHA
AHAHHA
AHAHHA
AHAHHA
AHAHHA
AHAHHA
AHAHHA
AHAHHA


In [441]:
Metat = create_class("MetaS", mro=fetch_typereferences(MetaS), attributes=deconstruct_class(MetaS))

print(Metat)

<class '__main__.MetaS'>


packager.py

In [442]:
import builtins
import os
from datetime import datetime
import inspect
from types import FunctionType, CodeType
from sys import builtin_module_names, modules

class Packer:
    def pack(self, obj: object, __globals__=globals()):
        self.metainfo = {}
        self.proceeded = []
        dump = self.dump(obj)
        if len(self.metainfo) == 0:
            return dump
        else:
            return {".META":self.metainfo, ".OBJ":dump}
    
    def funcdump(self, obj, isstatic= False):
        obj_id = id(obj)
        
        if isinstance(obj, staticmethod):
            return self.funcdump(obj.__func__, True)
        
        function_module = getattr(obj, "__module__", None)
        
        if function_module != None and function_module in builtin_module_names:
            self.metainfo.update({str(obj_id): {".metatype": "builtin func", ".name": obj.__name__, ".module": obj.__module__}})
        else:
            
            dumped = deconstruct_func(obj)

            code_dict = dumped[".code"]
            code_dict["co_code"] = [el for el in code_dict["co_code"]]
            code_dict["co_lnotab"] = [el for el in code_dict["co_lnotab"]]
            
            if self.metainfo.get(str(obj_id)) == None:
                self.metainfo.update({str(obj_id): {".code": self.dump(code_dict), ".metatype": "func", ".name": self.dump(dumped[".name"]), ".module": getattr(obj, "__module__", None), ".refs": self.dump(dumped[".references"]), ".defaults": self.dump(dumped[".defaults"])}})\

            return {".metaid": str(obj_id)}            
        
    def dump(self, obj: object):
        obj_id = id(obj)       
        
        if is_none(obj):
            return None
        
        if is_primitive(obj):
            return obj
        
        if type(obj) in [list, set, tuple, dict]:
            if isinstance(obj, dict):
                result = {key:self.dump(obj[key]) for key in obj}
            elif type(obj) in [set, tuple]:
                result = {".list":[self.dump(el) for el in obj], ".collection_type":f"{obj.__class__.__name__}"}
            else:
                result = [self.dump(el) for el in obj]
            return result
        
        if isinstance(obj, datetime):
            return {".time":str(obj.isoformat())}
        
        if obj_id in self.proceeded:
            return {".metaid": str(obj_id)}
        elif not getattr(obj, "__name__", None) in dir(builtins) :
            self.proceeded.append(obj_id)


        if inspect.ismodule(obj):
            try:
                if self.metainfo.get(str(obj_id)) == None:
                    if obj.__name__ in builtin_module_names:
                        self.metainfo.update({str(obj_id):{".metatype" : "module", ".name": obj.__name__}})
                    else:
                        self.metainfo.update({str(obj_id):{".code": get_code(obj), ".metatype" : "module", ".name": obj.__name__}})
            except Exception:
                self.metainfo.update({str(obj_id):{".metatype" : "module", ".name": obj.__name__}})
            return {".metaid": str(obj_id)}
        
        if getattr(obj, "__name__", None) and not is_basetype(obj):
            if obj.__name__ in dir(builtins):
                try:
                    self.proceeded.remove(str(obj_id))
                except Exception:
                    pass
                return {".metatype" : "builtin", ".builtin": obj.__name__}
            
            if inspect.ismethod(obj) or inspect.isfunction(obj) or isinstance(obj, staticmethod):
                return self.funcdump(obj)
            
            if inspect.isbuiltin(obj):
                self.metainfo.update({str(obj_id):{".metatype": "builtin-func", ".module": obj.__module__, ".name": obj.__name__}})
                return {".metaid": str(obj_id)}
            
            if is_instance(obj):
                type_, fields = deconstruct_instance(obj)
                type_id = id(type_)
                self.dump(type_)
                    
                data = {key: self.dump(fields[key]) for key in fields}
                return { ".metaid": str(type_id), ".fields": data}
            
            if inspect.isclass(obj):
                    
                mro = fetch_typereferences(obj)
                attrs = deconstruct_class(obj)
                mro = [self.dump(el) for el in mro]
                attrs = [self.dump((el[0], self.dump(el[1]), el[2])) for el in attrs]
                    
                if self.metainfo.get(str(obj_id)) == None:
                    self.metainfo.update({str(obj_id): {".metatype": "class", ".name": obj.__name__, ".module": getattr(obj, "__module__", None), ".class": {"mro":mro, "attrs":attrs}}})
                        
                return { ".metaid": str(obj_id)}
        else:
            if inspect.ismethod(obj) or inspect.isfunction(obj) or isinstance(obj, staticmethod):
                return self.funcdump(obj)
            
            if is_instance(obj):
                type_, fields = deconstruct_instance(obj)
                type_id = id(type_)
                self.dump(type_)
                    
                data = {key: self.dump(fields[key]) for key in fields}
                return { ".metaid": str(type_id), ".fields": data}

            return None
                        
         
        
class Unpacker:
    def unpack(self, src: object, __globals__=globals()):
        self._globals = __globals__
        if isinstance(src, dict): 
            if src.get(".META") != None and src.get(".OBJ") != None:
                self.metatypes ={}
                self.proceeded = []
                self.metadict = src[".META"]
                return self.load(src[".OBJ"])
            else:
                return self.load(src)
        
        if is_none(src):
            return None
        
        if is_primitive(src):
            return src
        
        if isinstance(src,list):
            return  [self.load(el) for el in src]
        
    def load(self, src, id_=None):
        if is_none(src):
            return None
        
        if is_primitive(src):
            return src
        
        elif isinstance(src,list):
            return [self.load(el) for el in src]
            
        
        elif isinstance(src, dict):
            if src.get(".metaid") != None and src.get(".metatype") == None:
                meta_id = src[".metaid"]
                obj = None
                
                if src[".metaid"] in self.proceeded:
                    obj = self.metatypes[meta_id]
                else:
                    obj = self.load(self.metadict[meta_id], meta_id)
                    self.metatypes[meta_id] = obj
                    self.proceeded.append(meta_id)
                if src.get(".fields"):
                    obj = create_instance(obj, self.load(src[".fields"]))
                return  obj
            
            elif src.get(".metatype"):
                metatype = src[".metatype"]
                
                if metatype == "func":
                    if src[".module"] != "__main__":
                        try:
                            exec(f'from {src[".module"]} import {src[".name"]}')
                            return eval(f'{src[".name"]}')
                        except Exception:
                            pass
                    
                    refs = self.load(src[".refs"])
                    nonlocals = refs[0]
                    globals_ = refs[1]
                    
                    co_raw = self.load(src[".code"])

                    co = CodeType(
                        co_raw["co_argcount"],
                        co_raw["co_posonlyargcount"],
                        co_raw["co_kwonlyargcount"],
                        co_raw["co_nlocals"],
                        co_raw["co_stacksize"],
                        co_raw["co_flags"],
                        bytes(co_raw["co_code"]),
                        co_raw["co_consts"],
                        co_raw["co_names"],
                        co_raw["co_varnames"],
                        co_raw["co_filename"],
                        co_raw["co_name"],
                        co_raw["co_firstlineno"],
                        bytes(co_raw["co_lnotab"]),
                        co_raw["co_freevars"],
                        co_raw["co_cellvars"]
                    )   

                    for el in globals_:
                        if el in globals().keys():
                            continue
                        else:
                            globals()[el] = self.load(globals_[el])
                    
                    closures = tuple(cell_factory(nonlocals[el]) for el in co.co_freevars)

                    naked = [
                        co,
                        globals(),
                        src[".name"],
                        src[".defaults"],
                        closures
                    ]

                    func = FunctionType(*naked)                    
                    return func
                
                if metatype == "builtin-func":
                    try:
                        exec(f'from {src[".module"]} import {src[".name"]}')
                        return eval(f'{src[".name"]}')
                    except Exception:
                        raise KeyError(f'builtin func "{src[".module"]}.{src[".name"]}" import failed')
                
                elif metatype == "class":
                    if src[".module"] != "__main__":
                        try:
                            exec(f'from {src[".module"]} import {src[".name"]}')
                            return eval(f'{src[".name"]}')
                        except Exception:
                            pass
                        
                    class_info = src[".class"]
                    mro = self.load(class_info["mro"])
                    cls = create_classbase(src[".name"], mro)

                    self.metatypes[id_] = cls
                    self.proceeded.append(id_)
                    
                    attrs = self.load(class_info["attrs"])
                    
                    
                        
                    return set_classattrs(cls, attrs)
                
                elif metatype == "module":
                    try:
                        exec(f'import {src[".name"]}')
                        result = eval(src[".name"])
                        return result
                    except Exception:
                        if ".code" in src.keys():
                            with open("{}/{}.py".format("/".join(modules["__main__"].__file__.split('/')[:-1]), src[".name"]), "w") as writer:
                                writer.write(src[".code"])
                            exec(f'import {src[".name"]}')
                            result = eval(src[".name"])
                            os.unlink("{}/{}.py".format("/".join(modules["__main__"].__file__.split('/')[:-1]), src[".name"]))
                            return result
                    raise KeyError(f'module"{src[".module"]}" import failed')
                
                elif metatype == "builtin":
                    if src.get(".builtin"):
                        return getattr(builtins, src[".builtin"])
                    else:
                        raise KeyError(f'builtin "{src[".builtin"]}" import failed')
                        
                else:
                    raise KeyError(f"Unexpected metatype: {metatype}")
            
            elif src.get(".collection_type"):
                if src[".collection_type"] == "tuple":
                    return tuple(el for el in self.load(src[".list"]))
                elif src[".collection_type"] == "set":
                    return set(el for el in self.load(src[".list"]))
                else:
                    return self.load(src[".list"])
            
            else:    
                res=  {
                    key: self.load(src[key]) for key in src
                }
                return res
            
            

In [443]:
# # print(deconstruct_func(Shitty.__init__))
# def unpack_func(obj):
#     code = obj["_code"]
#     exec(code)
#     func = eval(obj["_name"])
#     print(func.__code__.co_names)
#     stra = "ads"
#     globals()["globala"]= stra
#     return func

# # f = unpack_func(deconstruct_func(func1))
# # f("asd")
# print(json.dumps(_Packer().pack(json.dumps)))

In [444]:
def wrapper_(func):
    def wrap():
        print("boo!!!!")
        return func() 
    return wrap


func__ = wrapper_(func1)

print(func__.__closure__)

print(func__.__class__)

(<cell at 0x7f1bfd6e6100: function object at 0x7f1bfd45cc10>,)
<class 'function'>


In [445]:
# a = ClassA()

# print(a)



# print(json.dumps(_Packer().pack(a)))

In [446]:
# print(json.dumps(_Packer().pack(json.encoder)))

In [447]:
b = ClassA()
b.a = "dasda"

setattr(b, "sex", "Male")


print(json.dumps(Packer().pack(b)))


# inspect_element(json.encoder.encode_basestring)

{".META": {"139758185970992": {".code": {"co_argcount": 1, "co_posonlyargcount": 0, "co_kwonlyargcount": 0, "co_nlocals": 1, "co_stacksize": 3, "co_flags": 67, "co_code": [116, 0, 116, 1, 100, 1, 23, 0, 131, 1, 1, 0, 100, 0, 83, 0], "co_consts": {".list": [null, "Asdas"], ".collection_type": "tuple"}, "co_names": {".list": ["print", "globala"], ".collection_type": "tuple"}, "co_varnames": {".list": ["self"], ".collection_type": "tuple"}, "co_freevars": {".list": [], ".collection_type": "tuple"}, "co_cellvars": {".list": [], ".collection_type": "tuple"}, "co_filename": "<ipython-input-437-5bb8b7942706>", "co_name": "_getAttreasdasd", "co_firstlineno": 9, "co_lnotab": [0, 1]}, ".metatype": "func", ".name": "_getAttreasdasd", ".module": "__main__", ".refs": {".list": [{}, {"globala": "Hui"}, {"print": {".metatype": "builtin", ".builtin": "print"}}, {".list": [], ".collection_type": "set"}], ".collection_type": "tuple"}, ".defaults": null}, "36176656": {".metatype": "class", ".name": "Pare

In [464]:
aba = "HI "

def hello(name):
    return aba + name.upper()

str_ = hello

def wrapper_(func):
    def wrap():
        print("boo!!!!")
        return func() 
    return wrap


o = hello


# print(json.dumps(Packer().pack(o)))
v = Unpacker().unpack(json.loads(json.dumps(Packer().pack(o))))

print(o.__code__)
print(v.__code__)

hello(" gnida")
v(" suka")
# v = Unpacker().unpack(o)


<code object hello at 0x7f1bfd449df0, file "<ipython-input-464-8b74b3b3e970>", line 3>
<code object hello at 0x7f1bfd6b2c90, file "<ipython-input-464-8b74b3b3e970>", line 3>


'HI  SUKA'